# SparkSQL, Spark DataFrames, para Web Scraping

**Librerias Relavantes**

`SparkSQL` nos permitirá integrar fuentes de formato no estructurado a datos Tabulares.

`BeautifulSoup` nos permitirá de una manera simple hacer WebScraping.

`urlparse2` nos permitirá parsear una URL para obtener los datos necesarios para realizar analiis exploratorio.



Como siempre, empezamos obtniendo el `SparkContext`

In [1]:
import pyspark
from pyspark.sql import Row
sc = pyspark.SparkContext('local[*]')

In [2]:
sqlContext = pyspark.SQLContext(sc)

In [3]:
import urlparse2
from urlparse2 import urlparse

from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random
#instalar urlopen

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('display.mpl_style', 'default')
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (15, 5)

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


**Librerias Relavantes**

El Archivo `Link.raw` nos permitirá almacenar los Links Obtenidos por el WebScraping.

La primera ejecución el archivo no existira ya que se genera hasta tener el resultado del WebScraping.

El commando a continuación nos permite eliminar el archivo, esto si se requiere ejecutar el WebScraping a otra Pagina Objetivo coambiando a la ves el parametro  `urlTarget`



In [4]:
# define number as a set of words
! rm link.raw

rm: cannot remove ‘link.raw’: No such file or directory


**Párametro urlTarget**

Cambiar el URL Target cuando se quiera ejecutar el WebScraper para sacar `Link.raw` a donde se requiera.



In [ ]:
urlTarget="http://www.monster.com/"

In [ ]:
#Retrieves a list of all Internal links found on a page
def getInternalLinks(bsObj, includeUrl):
    includeUrl = urlparse(includeUrl).scheme+"://"+urlparse(includeUrl).netloc
    internalLinks = []
    #Finds all links that begin with a "/"
    for link in bsObj.findAll("a", href=re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith("/")):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

#Retrieves a list of all external links found on a page
def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    #Finds all links that start with "http" or "www" that do
    #not contain the current URL
    for link in bsObj.findAll("a", href=re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html)
    externalLinks = getExternalLinks(bsObj, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print("No external links, looking around the site for one")
        domain = urlparse(startingPage).scheme+"://"+urlparse(startingPage).netloc
        internalLinks = getInternalLinks(bsObj, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]

def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print("Random external link is: "+externalLink)
    c = urlparse(externalLink)
    with open("link.raw", "a") as link_log_c:
        link_log_c.write('"'+startingSite+'"|"'+str(c.scheme)+'"|"'+str(c.hostname)+'"|"'+str(c.port)+'"|"'+str(c.path)+'"|"'+str(c.username)+'"|"'+str(c.params)+'"\n')
    followExternalOnly(externalLink)

In [ ]:
#Collects a list of all external URLs found on the site
allExtLinks = set()
allIntLinks = set()
cont=0
def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = urlparse(siteUrl).scheme+"://"+urlparse(siteUrl).netloc
    bsObj = BeautifulSoup(html)
    internalLinks = getInternalLinks(bsObj,domain)
    externalLinks = getExternalLinks(bsObj,domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print("Random external link is: "+link)
            o = urlparse(link)
            with open("link.raw", "a") as link_log_o:
                link_log_o.write('"'+siteUrl+'"|"'+str(o.scheme)+'"|"'+str(o.hostname)+'"|"'+str(o.port)+'"|"'+str(o.path)+'"|"'+str(o.username)+'"|"'+str(o.params)+'"\n')
                return link      
           
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            getAllExternalLinks(link)
   

**IMPORTANTE**

La siguiente secion de codigo ejecuta `WebScraping` nos permitirá almacenar los Links contenidos en la paguina Web objetivo.

La primera ejecución puede marcar errores, sin embargo, el error esta controlado "casi todo!!", en caso de algún error ejecutar nuevamente hasta llegar a un numero de Links mayor a 100% (**como sigerencia para que la visualización sea adecuada**).`



In [ ]:
pages = set()
random.seed(datetime.datetime.now())

try:
    followExternalOnly(urlTarget)
    llIntLinks.add(urlTarget)
    getAllExternalLinks(urlTarget)
    
except ConnectionResetError as a:
    print(a)
except ValueError as b:
    print(b)
except KeyboardInterrupt as c:
    print(c)
except NameError as d:
    print(d)
except ValueError as f:
    print(f)
except urllib2.HTTPError as err:
   if err.code == 404:
       print("Execption on Scraping process, try again!!")
   else:
       raise


## Validamos El Scrapin de Links

Ahora leeremos el archivo de Link.raw creado por el Scraping

In [ ]:
! wc -l link.raw

**NOTA**: Aqui validamos que se este generando el archivo con los Links Scrapeados, el número adecuado debe ser mayor a 100, esto se logra al rededor de 7 a 10 ejecuciones, **esto dependerá de la configuración pagina que se esta scrapeando**.


## RDD desde los datos Crudos (Stage 2)

Creamos **Stage #2** generamos la extracción de los Links identificados pasando de RDD a SQLContext, esto para presentar la inforamación de forma tabular.

In [ ]:
def getTransaccion(linea):
    cells = linea.split('|')
    cells[6] = str(cells[6])
    return Transaccion(*cells)

Creamos **Stage #2** reutilizamos funcion desarrollda en la clase DPA para lectura y formate del schema para RDD.

In [ ]:
lnk_rdd = sc.textFile("link.raw")
Transaccion = Row('siteUrl','scheme','hostname','port','path','username','params')
txs = lnk_rdd.map(getTransaccion)
txs_df = txs.toDF()
txs_df.registerTempTable('Link')
pd_tbl_EASentiment = sqlContext.sql('select siteUrl, scheme, hostname, port, path, username, params from Link').toPandas()
pd_tbl_EASentiment.head(15)

## DF y SQLContext (Stage 3)

Se trabaja con la información ahora en forma tabular para tener los datos **consolidados**

# Analisis de Datos con Pandas

## Links Relacionados a la Paguina Web Scrapeada

-Se extraen los links internos y externos en lapagina Web que se esta **análizando**.


In [ ]:
num_links = sqlContext.sql('select hostname, count(hostname) as total_hostname from Link group by hostname order by total_hostname desc').toPandas()
links_by_hostname_pd = num_links.head(15)
links_by_hostname_pd=links_by_hostname_pd.set_index(['hostname'])

links_by_hostname_pd.plot(kind='barh',fontsize=15)
# Etiquetas de la tabla
plt.xlabel('Número de Hostname')
plt.ylabel('Paginas Web')
plt.title('Top 15 de Paginas que tienen Link con Hostname',)


- Verificar que poguina tienen mas Links con el Hostname **(Top 15)**



In [ ]:
projects_by_sentiment_pd = sqlContext.sql('select hostname, count(hostname) as total_hostname from Link group by hostname order by total_hostname desc').toPandas()
projects_by_sentiment_pd = projects_by_sentiment_pd.head(10)
projects_by_sentiment_pd=projects_by_sentiment_pd.set_index(['hostname'])
# Propiedades de Plot
plt.rcParams['figure.figsize'] = (25, 25)
projects_by_sentiment_pd.plot(kind='pie',subplots=True,autopct='%.2f',fontsize=15)
plt.title('Proporcion de Paginas',fontsize=25)


- Proporcion de Paguinas que tienen realación con el Hostname Scrapeado
- Del grafico anterior, se muestran las categorias mas relevante.
- Lo anterior tambien marca un dato curioso que es la ** la correlación **, de contenido entre la URL con los Links a otras paguinas  **podemos inferir calidad de contenido**.
